In [57]:
import os
import sys
import random
import numpy as np

from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

from openff.units.elements import SYMBOLS
from openff.toolkit import Molecule
from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings, GridSettingsType, GridGenerator
from openff.recharge.utilities.molecule import smiles_to_molecule
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
from openff.units import unit
from qcelemental.models.common_models import Model

sys.path.append('/Users/localadmin/Documents/projects/QM_ESP_Psi4')

from source.optimize.openff_psi4_gen import Psi4Generate
from source.conformers.conformer_gen import Conformers
from source.utilities.conversion_functions import conf_to_xyz_string


import psi4



In [12]:
#load our molecule to create the conformers

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

In [13]:
#test the optimizer

optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
optimize_MMFF



0

In [14]:
#test all the conformers

for id in range(rdmol.GetNumConformers()):
    optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
    print(f'conformer:{id}, optimize (0 = converge, 1 = more iterations req): {optimize_MMFF}')

conformer:0, optimize (0 = converge, 1 = more iterations req): 0
conformer:1, optimize (0 = converge, 1 = more iterations req): 0
conformer:2, optimize (0 = converge, 1 = more iterations req): 0
conformer:3, optimize (0 = converge, 1 = more iterations req): 0
conformer:4, optimize (0 = converge, 1 = more iterations req): 0
conformer:5, optimize (0 = converge, 1 = more iterations req): 0
conformer:6, optimize (0 = converge, 1 = more iterations req): 0
conformer:7, optimize (0 = converge, 1 = more iterations req): 0
conformer:8, optimize (0 = converge, 1 = more iterations req): 0
conformer:9, optimize (0 = converge, 1 = more iterations req): 0


In [15]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol


ModuleNotFoundError: No module named 'py3Dmol'

In [ ]:
#do the conformers look structurally sensible

for id in range(rdmol.GetNumConformers()):
    IPythonConsole.drawMol3D(rdmol,confId=id)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
optimize_MMFF

0

In [ ]:
#now create a series of xyz files so we can run the conformers with ASE, first convert our conformers to xyz format and then write to file

from openff.units import unit

cwd = os.getcwd()

conformers = []



for confs in range(rdmol.GetNumConformers()):
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


for number ,conformer in enumerate(conformers):
    atoms = [
            {
                "element": SYMBOLS[atom.atomic_number],
                "x": conformer[index, 0],
                "y": conformer[index, 1],
                "z": conformer[index, 2],
            }
            for index, atom in enumerate(molecule.atoms)
        ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    try:
        f = open(cwd+f"/conformer_{number}.xyz", 'x')
        f.write(xyz)
        f.close()
    except FileExistsError:
        continue

In [ ]:


CWD = os.getcwd()

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Setup geometry optimisation
hf_model = Model(method="hf", basis="6-31G*")
spec = QCInputSpecification(model=hf_model, keywords={}, driver="gradient")
opt_spec = OptimizationInput(
            initial_molecule=qc_mol,
            input_specification=spec,
            keywords={"coordsys": "dlc", 
                      "program": "psi4"}                                        
        )

opt = qcengine.compute_procedure(opt_spec, "geometric")
print(f'geometry optimiztion was {opt.error}')
#return optimized molecule
optmized_mol = opt.final_molecule

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )

grid = GridGenerator.generate(test_mol, optmized_mol.geometry*unit.angstrom, grid_settings)

grid = grid.to(unit.angstrom).m
np.savetxt("grid.dat", grid, delimiter=" ", fmt="%16.10f")
#compute one-electron properties.
opt_input_2 = { "molecule" : optmized_mol,
              "driver" : "energy",
              "model" : {"method":"scf","basis":"6-31G*"},
              "protocols":{"wavefunction":"all","stdout":True,"native_files":"all"},
              "keywords":{"scf_properties":["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"]}                               
              }


opt_2 = qcengine.compute(opt_input_2, "psi4", task_config={"scratch_directory":CWD,"scratch_messy":True})

print(opt_2.dict())

esp = (np.loadtxt("grid_esp.dat").reshape(-1, 1) * unit.hartree / unit.e)

electric_field = (np.loadtxt("grid_field.dat")* unit.hartree/ (unit.e * unit.bohr))

rdkit has generated 10 conformers from a requested 10 conformers
geometry optimiztion was None


/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


{'id': None, 'input_data': {'molecule': {'schema_name': 'qcschema_molecule', 'schema_version': 2, 'validated': True, 'symbols': array(['H', 'O', 'H'], dtype='<U1'), 'geometry': array([[-1.42427673, -0.3639913 ,  0.        ],
       [-0.00142791,  0.72236003,  0.        ],
       [ 1.42570464, -0.35836872,  0.        ]]), 'name': 'H2O', 'molecular_charge': 0.0, 'molecular_multiplicity': 1, 'connectivity': [(0, 1, 1.0), (1, 2, 1.0)], 'fix_com': True, 'fix_orientation': True, 'provenance': {'creator': 'QCElemental', 'version': '0.26.0', 'routine': 'qcelemental.molparse.from_schema'}, 'extras': {'canonical_isomeric_explicit_hydrogen_mapped_smiles': '[H:1][O:2][H:3]'}}, 'driver': 'energy', 'model': {'method': 'scf', 'basis': '6-31G*'}, 'protocols': {'wavefunction': 'all', 'stdout': True, 'native_files': 'all'}, 'keywords': {'scf_properties': ['GRID_ESP', 'GRID_FIELD', 'MULLIKEN_CHARGES', 'LOWDIN_CHARGES', 'DIPOLE', 'QUADRUPOLE', 'MBIS_CHARGES']}, 'provenance': {'cpu': 'Intel(R) Xeon(R) CPU 

In [ ]:
psi4.core.variables(['MBIS Charges'])

{'CURRENT ENERGY': -76.00688623049868,
 'CURRENT REFERENCE ENERGY': -76.00688623049868,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232170089,
 'HF POTENTIAL ENERGY': -151.71896855219958,
 'HF TOTAL ENERGY': -76.0068862304987,
 'HF VIRIAL RATIO': 2.003893749844908,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060212095,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623049868,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623049868,
 'TWO-ELECTRON ENERGY': 37.679255168868295,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78959587e+00, -8.82137120e-04,  0.00000000e+00],
        [-8.82137120e-04, -4.43606969e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'SCF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ])}

In [16]:
from source.optimize.openff_psi4_gen import Psi4Generate
import psi4

test_mol =  smiles_to_molecule('[H]O[H]')

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )


esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

xyz = Psi4Generate.run_calc(test_mol,conformer_list[0],esp_settings,compute_esp=True,compute_field=True, minimize=False)

molecule = psi4.geometry(
"""
3
[H]O[H]
H	-0.81024874	-0.18571238	-0.0
O	-0.00248133	0.36959931	-0.0
H	0.81273007	-0.18388693	0.0
"""
)

molecule.set_molecular_charge(0)
molecule.set_multiplicity(1)

E, wfn = psi4.energy('hf/6-31g*', return_wfn=True, molecule = molecule)
psi4.prop('hf/6-31G*', properties=["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"])
#psi4.core.print_variables()
psi4.core.variables(['MBIS Charges'])
molecule.fcharge

rdkit has generated 10 conformers from a requested 10 conformers


UnboundLocalError: cannot access local variable 'grid' where it is not associated with a value

In [77]:
import os
import sys
import random
import numpy as np

from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

from openff.units.elements import SYMBOLS
from openff.toolkit import Molecule
from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings, GridSettingsType, GridGenerator
from openff.recharge.utilities.molecule import smiles_to_molecule
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
from openff.units import unit
from qcelemental.models.common_models import Model

sys.path.append('/Users/localadmin/Documents/projects/QM_ESP_Psi4')

from source.optimize.openff_psi4_gen import Psi4Generate
from source.conformers.conformer_gen import Conformers
from source.utilities.conversion_functions import conf_to_xyz_string


import psi4

sys.settrace 

test_mol =  smiles_to_molecule('[H]O[H]')

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )
grid = GridGenerator.generate(test_mol, conformer_list[0], grid_settings)

grid = grid.to(unit.angstrom).m
np.savetxt("grid.dat", grid, delimiter=" ", fmt="%16.10f")


esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

xyz = conformer_string = conf_to_xyz_string(conformer_list[0], test_mol)

psi4.set_output_file('output.dat')

molecule = psi4.core.Molecule.from_string("""
                                            nocom
                                            noreorient
                                            H	-0.81024874	-0.18571238	-0.0
                                            O	-0.00248133	0.36959931	-0.0
                                            H	0.81273007	-0.18388693	0.0 
                                          """)

molecule.set_molecular_charge(0)
molecule.set_multiplicity(1)

psi4.set_options({"DFT_SPHERICAL_POINTS":"590",
                  "DFT_RADIAL_POINTS":"99"})

E, wfn = psi4.prop('hf/6-31G*', properties=["GRID_ESP", 
                                            "GRID_FIELD",
                                            "MULLIKEN_CHARGES", 
                                            "LOWDIN_CHARGES", 
                                            "DIPOLE", 
                                            "QUADRUPOLE", 
                                            "MBIS_CHARGES"],
                                            molecule = molecule, 
                                            return_wfn = True,
                                            )  #"GRID_ESP", "GRID_FIELD"
psi4.core.variables(['MBIS Charges'])
wfn.variables()

rdkit has generated 10 conformers from a requested 10 conformers


{'CURRENT ENERGY': -76.00688623048389,
 'CURRENT REFERENCE ENERGY': -76.00688623048389,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232169239,
 'HF POTENTIAL ENERGY': -151.71896855217628,
 'HF TOTAL ENERGY': -76.00688623048389,
 'HF VIRIAL RATIO': 2.0038937498448255,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060210559,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623048389,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623048389,
 'TWO-ELECTRON ENERGY': 37.67925516886774,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78962776e+00,  4.75167568e-03,  0.00000000e+00],
        [ 4.75167568e-03, -5.40692800e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'LOWDIN CHARGES': array([ 0.36108223, -0.72231233,  0.3612301 ]),
 'MBIS CHARGES': array([[ 0.46457588],
        [-0.92803681],
        [ 0.46346084]]),
 

In [39]:
prop

-76.00688623049402

In [51]:
wfn.variables()

{'CURRENT ENERGY': -76.00688623049402,
 'CURRENT REFERENCE ENERGY': -76.00688623049402,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232169566,
 'HF POTENTIAL ENERGY': -151.71896855218966,
 'HF TOTAL ENERGY': -76.00688623049402,
 'HF VIRIAL RATIO': 2.0038937498449156,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060211433,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623049402,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623049402,
 'TWO-ELECTRON ENERGY': 37.67925516886635,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78959587e+00, -8.82137118e-04,  0.00000000e+00],
        [-8.82137118e-04, -4.43606969e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'LOWDIN CHARGES': array([ 0.36108223, -0.72231233,  0.3612301 ]),
 'MBIS CHARGES': array([[ 0.46457588],
        [-0.92803681],
        [ 0.46346084]]),
 

In [72]:
molecule = psi4.core.Molecule.from_string("""
                                            nocom
                                            noreorient
                                            H	-0.81024874	-0.18571238	-0.0
                                            O	-0.00248133	0.36959931	-0.0
                                            H	0.81273007	-0.18388693	0.0 
                                          """)

molecule.to_string('xyz')


'3\n0 1 H2O\nH                    -0.810248740000    -0.185712380000     0.000000000000\nO                    -0.002481330000     0.369599310000     0.000000000000\nH                     0.812730070000    -0.183886930000     0.000000000000\n'

In [89]:
mulliken_charges = wfn.variable("MULLIKEN_CHARGES")
lowdin_charges = wfn.variable("LOWDIN_CHARGES")
dipole = wfn.variable("HF DIPOLE")
quadrupole = wfn.variable("HF QUADRUPOLE")
mbis_charges = wfn.variable("MBIS CHARGES")

/var/folders/5m/c625w21d6q3cgv41hc6yjs100000gq/T/ipykernel_24369/3717132043.py:1: FutureWarning: Using QCVariable `MULLIKEN_CHARGES` instead of `MULLIKEN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  mulliken_charges = wfn.variable("MULLIKEN_CHARGES")
/var/folders/5m/c625w21d6q3cgv41hc6yjs100000gq/T/ipykernel_24369/3717132043.py:2: FutureWarning: Using QCVariable `LOWDIN_CHARGES` instead of `LOWDIN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  lowdin_charges = wfn.variable("LOWDIN_CHARGES")


In [90]:
variable_names = ["MULLIKEN_CHARGES", "LOWDIN_CHARGES", "HF DIPOLE", "HF QUADRUPOLE", "MBIS CHARGES"]
variables = {name: wfn.variable(name) for name in variable_names}

/var/folders/5m/c625w21d6q3cgv41hc6yjs100000gq/T/ipykernel_24369/462452538.py:2: FutureWarning: Using QCVariable `MULLIKEN_CHARGES` instead of `MULLIKEN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  variables = {name: wfn.variable(name) for name in variable_names}
/var/folders/5m/c625w21d6q3cgv41hc6yjs100000gq/T/ipykernel_24369/462452538.py:2: FutureWarning: Using QCVariable `LOWDIN_CHARGES` instead of `LOWDIN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  variables = {name: wfn.variable(name) for name in variable_names}


In [94]:
from qcelemental.models import Molecule as QCMol
qc_mol =  test_mol.to_qcschema(conformer=0)
qc_mol = QCMol(**qc_mol.dict(exclude={"fix_com", "fix_orientation"}), fix_com=True, fix_orientation=True)
molecule = psi4.geometry(qc_mol.to_string("psi4"))
qc_mol.geometry

array([[-1.53114822, -0.35094553,  0.        ],
       [-0.00468903,  0.69844148,  0.        ],
       [ 1.53583725, -0.34749594,  0.        ]])

In [1]:
import sys
sys.path.append('/Users/localadmin/Documents/projects/QM_ESP_Psi4')

from source.inputSetup.SmilesInputs import ReadInput
from source.optimize.esp_generator_wrapper import ESPGenerator, PropGenerator
from source.conformers.conformer_gen import Conformers
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.esp import ESPSettings



    
#Read the .smi input and add to list
smiles = ReadInput.read_smiles('test_files.smi')

# Define the grid that the electrostatic properties will be trained on and the
# level of theory to compute the properties at.
grid_settings = LatticeGridSettings(
    type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
)

esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

#Loop through molecules
for mol in smiles:
    molecule = smiles_to_molecule(mol)
    #Generate the conformers
    conformer_list = Conformers.generate(molecule, generation_type='rdkit')
    ESP_gen = PropGenerator(molecule = molecule, conformers = conformer_list, esp_settings = esp_settings, grid_settings = grid_settings)
    print(f'number of cores is {ESP_gen.ncores}')
    print(f'memory is {ESP_gen.memory}')
    ESP_gen.run_props()
    #ESP_gen.fetch_data()

/Users/localadmin/miniconda3/envs/openff/lib/python3.11/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


rdkit has generated 10 conformers from a requested 10 conformers
number of cores is 4
memory is 0.739
conformer 0 for [H][O][C]([H])([H])[C]([H])([O][H])[C]([H])([H])[O][H]
FailedOperation(error=ComputeError(error_type='unknown', error_message='geomeTRIC run_json error:\nTraceback (most recent call last):\n  File "/Users/localadmin/miniconda3/envs/openff/lib/python3.11/site-packages/geometric/run_json.py", line 258, in geometric_run_json\n    geometric.optimize.Optimize(coords, M, IC, engine, dirname, params)\n  File "/Users/localadmin/miniconda3/envs/openff/lib/python3.11/site-packages/geometric/optimize.py", line 779, in Optimize\n    return optimizer.optimizeGeometry()\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/Users/localadmin/miniconda3/envs/openff/lib/python3.11/site-packages/geometric/optimize.py", line 692, in optimizeGeometry\n    self.calcEnergyForce()\n  File "/Users/localadmin/miniconda3/envs/openff/lib/python3.11/site-packages/geometric/optimize.py", line 283, in c

AttributeError: 'FailedOperation' object has no attribute 'final_molecule'

In [1]:
from xtb.interface import Calculator


ImportError: xtb C extension unimportable, cannot use C-API

In [8]:
!mamba install -c conda-forge xtb


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.2) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['xtb']

warning  libmamba Cache file "/Users/localadmin/mambaforge/pkgs/cache/31ce02e0.json" was mo